In [ ]:
import os
import h5py 
import torch 
import numpy as np
from torch.nn.functional import interpolate
from pathlib import Path

input_dir = '/home/mim-server/datasets/pushT/224/'
output_dir = '/home/mim-server/datasets/pushT/sharded/'
input_files = os.listdir(input_dir)
shard_file = input_files[0]  

input_data_path = os.path.join(input_dir, shard_file)
output_dir = Path(output_dir)
output_dir.mkdir(parents=True, exist_ok=True)

def loadPushTData(data_path):
    with h5py.File(data_path, 'r') as f:
        data = f
        actions = data['actions'][:]
        images = data['cam1'][:]
    images = torch.from_numpy(images).float()
    images = interpolate(images, size=(256,256), mode='bilinear', align_corners=False)
    images = images.permute(0, 2, 3, 1)  # NHWC to NCHW
    images = (images*255).to(torch.uint8)
    return actions[None], images[None].numpy()

def saveShardedData(output_path, actions, images):
    with h5py.File(output_path, 'w') as f:
        comp_args = {'compression': 'gzip', 'compression_opts': 4}
        image_shape = images.shape[2:]
        action_shape = actions.shape[2:]
        images_ds = f.create_dataset('images', shape=images.shape, dtype=np.uint8, chunks=(1, 128, *image_shape), **comp_args)
        actions_ds = f.create_dataset('actions', shape=actions.shape, dtype=np.float32, chunks=(1, 128, *action_shape), **comp_args)
        f.create_dataset('episode_lengths', data=np.array([images.shape[1]], dtype=np.int32))
        f.attrs['num_episodes'] = images.shape[1]
        images_ds[:] = images
        actions_ds[:] = actions

In [ ]:
for i, data_file in enumerate(input_files):
    print(f"Processing file {i+1}/{len(input_files)}: {data_file}")
    input_data_path = os.path.join(input_dir, data_file)
    output_data_path = os.path.join(output_dir, f"shard_{i:04d}.h5")
    actions, images = loadPushTData(input_data_path)
    print(f"  Loaded data shape - actions: {actions.shape}, images: {images.shape}")
    saveShardedData(output_data_path, actions, images)

In [ ]:
str(output_dir)

In [ ]:
from dreamerv4.datasets import ShardedHDF5Dataset

dataset = ShardedHDF5Dataset(str(output_dir), window_size=192, stride=1)

In [1]:
import torch 
from dreamerv4.datasets import ShardedHDF5Dataset
from dreamerv4.models.utils import load_tokenizer
from dreamerv4.models.utils import load_denoiser
from dreamerv4.models.dynamics import DenoiserWrapper
from dreamerv4.models.tokenizer import TokenizerWrapper
from hydra import initialize, compose
from dreamerv4.loss import ForwardDiffusionWithShortcut, compute_bootstrap_diffusion_loss
from omegaconf import OmegaConf
with initialize(version_base=None, config_path="scripts/config"):
    cfg = compose(config_name="dynamics/pushT.yaml")

model = DenoiserWrapper(cfg)
diffuser = ForwardDiffusionWithShortcut(num_noise_levels=cfg.denoiser.num_noise_levels)
# model = TokenizerWrapper(cfg)

/home/mim-server/miniconda3/envs/dreamerv4/lib/python3.11/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'dynamics/pushT.yaml': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


In [2]:
cfg.denoiser

{'dual_stream': False, 'num_action_tokens': 1, 'num_latent_tokens': 256, 'num_register_tokens': 4, 'context_length': 32, 'max_sequence_length': 32, 'model_dim': 768, 'latent_dim': 32, 'n_layers': 4, 'n_heads': 12, 'n_kv_heads': 12, 'dropout_prob': 0.0, 'qk_norm': False, 'num_noise_levels': 128, 'n_actions': 2}

In [3]:
sample_latent = torch.randn(1, 16, cfg.denoiser.num_latent_tokens, cfg.denoiser.latent_dim)
sample_action = torch.randn(1, 16, cfg.denoiser.n_actions)
info = diffuser(sample_latent, sample_action)

In [5]:
compute_bootstrap_diffusion_loss(info, model)

(tensor(1.1406, grad_fn=<DivBackward0>),
 tensor(0.0154, grad_fn=<DivBackward0>),
 tensor(0.9788, grad_fn=<DivBackward0>),
 tensor(0.0156, grad_fn=<DivBackward0>))

In [ ]:
import torch

ckpt_path = "/home/mim-server/projects/rooholla/dreamer-v4-draft/checkpoints/dynamics_ckpts/video-pushT-110M.pt"
state = torch.load(ckpt_path, map_location='cpu')
if 'dyn' in state.keys():
    sd = state['dyn']
else:
    sd = state['model']
    
to_delete = []
clean_sd = {k.replace("_orig_mod.", ""): v for k, v in sd.items()}
for k in clean_sd.keys():
    if k.endswith("cos_emb") or k.endswith("sin_emb") or "temporal_mask_full" in k:
        to_delete.append(k)
for k in to_delete:
    del clean_sd[k]

spell_corrrected = {}
for key in clean_sd.keys():
    val = clean_sd[key]
    if 'diffuion' in key:
        key = key.replace('diffuion', 'diffusion')
        print(key)

    spell_corrrected[key]=val

clean_sd = spell_corrrected

state['model'] = clean_sd
if 'model' not in state.keys():
    clean_sd = {'model.'+k: v for k, v in clean_sd.items()}
    state['model'] = clean_sd
if 'dyn' in state.keys():
    del state['dyn']



print(state.keys())
print(state['model'].keys())

In [ ]:
model.load_state_dict(state['model'], strict=True)


In [ ]:
torch.save(state, ckpt_path)